In [1]:
import json
import re

In [2]:
with open("intermediate_outputs/formatted_appState_w_dependencies.json", 'r') as f:
    data = json.load(f)

# build graph using dot notation

In [3]:
def build_digraph(keep: list[str]):
    global data;
    node_map = {key: index for index, key in enumerate(keep)}
    colors_map = {
        "widget": ["gainsboro", "black"],
        "datasource": ["#BBD686", "black"],
        "function": ["#F2B5D4", "black"],
        "frame": ["gold2", "black"],
    }

    dot_data = "digraph G {\n"\
    "  node [shape=record, style=filled]\n"\
    "  edge [color=black]\n"
    for name, index in node_map.items():
        fill_color = "yellow"
        text_color = "black"
        if name in data.keys() and "type" in data[name].keys():
            [fill_color, text_color] = colors_map.get(data[name].get("type"))
            if len([d for d in (data[name]["component_relations"].get("dependants") or []) if d["type"] != "widget"]) == 0:
                text_color = "white"
            if data[name].get("type") == "widget":
                relations = data[name]["component_relations"]
                if len(relations.get("dependants") or []) == 0:
                    fill_color = "black"
                    text_color = "white"
            name = f"{{{name}|{{{data[name].get('type')}|{data[name].get('subtype')}}}}}"
        dot_data += f'  {index} [label="{name}" fillcolor="{fill_color}" fontcolor="{text_color}"];\n'
    dot_data += "\n"


    method_map = {
        "update": {
            "label": "U",
            "color": "deepskyblue",
            "penwidth": "1.5"
        },
        "trigger": {
            "label": "T",
            "color": "red",
            "penwidth": "3"
        },
    }
    def get_method_styles(method):
        color = "gray50"
        penwidth = "1"
        if method in method_map:
            color = method_map.get(method)["color"]
            penwidth = method_map.get(method)["penwidth"]

        return f'['\
            f' color="{color}"'\
            f' penwidth="{penwidth}"'\
            f']'

    links = set([
        f'{node_map.get(d["triggered_by"])} -> {node_map.get(d["pluginId"])} {get_method_styles(d["method"])}'
        for value 
        in data.values() 
        for d 
        in (value["component_relations"].get("dependants") or [])
        if d["pluginId"] in node_map and d["triggered_by"] in node_map
    ])
    dot_data += "\n".join(links)
    dot_data += "\n}"

    return dot_data

In [4]:
keep_a = [ 
    key 
    for key 
    in data.keys() 
    if "dependants" in data[key]["component_relations"] 
        and len(data[key]["component_relations"]["dependants"]) > 0  
]
keep_b = [ 
    id
    for value 
    in data.values() 
    for id
    in [
        dependant["pluginId"]
        for dependant
        in (value["component_relations"].get("dependants") or [])
    ]
]

In [5]:
keep_all = [
    name 
    for name 
    in set(keep_a + keep_b) 
    if not not name
    and name in data
    and data[name].get("type") != "setting"
    # and not (
    #     data[name].get("type") == "widget"
    #     and len((data[name]["component_relations"].get("dependants") or [])) == 0
    # )
]

with open("outputs/full_dependency_graph.dot", 'w') as f:
    f.write(build_digraph(keep_all))

In [6]:
keep_queries = [
    name 
    for name 
    in set(keep_a + keep_b) 
    if not not name
    and name in data
    and data[name].get("type") != "setting"
    and data[name].get("type") != "widget"
    # and not (
    #     data[name].get("type") == "widget"
    #     and len((data[name]["component_relations"].get("dependants") or [])) == 0
    # )
]

with open("outputs/queries_only_dependency_graph.dot", 'w') as f:
    f.write(build_digraph(keep_queries))

In [7]:
keep_queries_except_employee = [
    name
    for name
    in keep_queries
    if name != 'employee'
]

with open("outputs/minus_employee.dot", 'w') as f:
    f.write(build_digraph(keep_queries_except_employee))